
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <meta http-equiv="Content-Style-Type" content="text/css" />
    <meta name="generator" content="pandoc" />
    <title></title>
    <style type="text/css">code{white-space: pre;}</style>
  </head>
  

# MACHINE LEARNING FOR RESEARCHERS

# Notebook 5. Redes Neuronales Artificiales


    
Este notebook presenta una introducción a las redes neuronales artificiales. Se empleará la librería de alto nivel **Keras**, que empleará **Tensorflow** para crear los grafos asociados a las redes. Como ejemplos demostrativos se abordarán los ejemplos de base mostrados en el Notebook 1 (regresión y clasificación).

## Redes Neuronales

Una red neuronal artificial **(Artificial Neural Network - ANN)** es una
  estructura que permite crear modelos de predicción **no lineales**. Se
  emplean tanto en aprendizaje supervisado como no supervisado. En aprendizaje
  supervisado permiten implementar sistemas regresores (incluidos multi-target),
  y clasificadores probabilísticos (binarios o multiclase).
  
Al crear modelos no lineales permiten usar **bases con un número fijo de
  características** pero que serán **adaptativas**, i.e., que se ajustarán
  automáticamente a los datos de entrenamiento. Como penalización, resultan
  **funciones de coste no convexas**, por lo que hay que desarrollar
  algoritmos de entrenamiento específicos, como **back-propagation**.
  
  La red posee una **capa de entrada**, con $D$ entradas, un por cada variable de la instancia $\pmb{x}$.  A continuación hay $L$ capas, cada una con $M(l)$ nodos.  Las $L-1$ primeras capas se llaman **capas ocultas**, tras ella está la **capa de salida**, con tantas salidas $K$ como dimensiones tenga el **target** a predecir.

  Cada una de los nodos de las capas se denomina **neurona**, por analogía con las redes neuronales biológicas. La salida del nodo $i$ de la capa $l$ se denota $z^{(l)}_i$, o simplemente $x_i$ o $y_i$ para las capas de entrada y de salida, respectivamente.  Se asume además un nodo extra $x_0$=1, y $z^{(l)}_0$=1 para permitir un término de bias. 
  
  Las redes se identifican según el número de $L$-capas. 
  
 De una capa a la siguiente, la salida del nodo $i$ se pondera por un peso
  $w^{(l)}_{ji}$, siendo $l$ la capa y $j$ el nodo de la capa $l$.

  La suma ponderada por los pesos de las entradas a un nodo se llama **activación**. Para
  el nodo $j$ de la capa $l$ viene dado por:
  
  \begin{equation*}
  a^{(l)}_j = \sum_{i=0}^{M(l)} w^{(l)}_{ji} z_i
  \end{equation*}

  Esa entrada se transforma usando una **función de activación** $h(\cdot)$ para generar la salida
  hacia la siguiente capa:
  
  \begin{equation*}
  z^{(l)}_j = h(a^{l}_j) = h\left(\sum_{i=0}^{M(l)} w^{(l)}_{ji} z_i\right)
  \end{equation*}

  La función de activación dependerá de la capa (o nodo) y se escoge **no-lineal y diferenciable**.
 
  
  A continuación crearemos una red de 2-capas con $M$ neuronas en la capa oculpa para tratar el problema base de regresión planteado en el Notebook 1. **Al tratarse de un problema de regresión se escoge como activación en la última capa la función identidad**.


### Problemas de regresión

In [ ]:
# Bibliotecas necesarias
import numpy as np
import os

# Configuramos matplotlib
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Semilla fija para reproducibilidad
np.random.seed(1)

# Generamos data set
X = np.linspace(0,1,num=10) # Tipo lista
X = np.array([X]).T # Lo pasamos a vector "columna" en numpy (tipo ndarray)
t = np.sin(2*np.pi*X) + 0.1*np.random.randn(10, 1) 
xfull = np.linspace(0,1,num=100);
xfull = np.array([xfull]).T 
treal = np.sin(2*np.pi*xfull);

# Data set plotting
plt.plot(X, t, "b.")
plt.plot(xfull, treal, "g-")
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$t$", rotation=0, fontsize=18)
plt.axis([0, 1, -1.5, 1.5])
plt.show()

In [ ]:
 # Entrenamos un autoencoder construido en keras
import keras

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

M=3
model = keras.models.Sequential([
    keras.layers.Dense(M, activation="tanh", input_shape=X.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X, t, epochs=100000, batch_size=2) # Calcula los pesos óptimos dado el training set X,t

y = model.predict(xfull)  # Precide nuevos puntos

plt.plot(xfull, y, "r-")
plt.plot(xfull, treal, "g-")
plt.plot(X, t, "b.")
plt.axis([0, 1, -1.5, 1.5])
plt.show()

Observe como el tiempo de entrenamiento es **mucho mayor** que con el regresor logístico. La ventaja es que en ningún momento debe crearse una matriz de diseño, **¡¡la red neuronal se ocupa de encontrar la mejor transformación al espacio de características!!**.

### Problemas de clasificación

Vamos a aplicar ahora una ANN para la clasificación del data set Esfericos vs. Toroidales.

In [ ]:
np.random.seed(1)
r = np.random.rand(50, 1) #0...1
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_0 = np.c_[r*np.cos(a), r*np.sin(a)]

r = 0.9 + np.random.rand(50, 1) #0.9...1.9
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_1 = np.c_[r*np.cos(a), r*np.sin(a)]

X = np.r_[X_0, X_1]
t = np.r_[np.zeros([50,1]),np.ones([50,1])]

# Plot data set
plt.figure(figsize=(6, 6))
plt.plot(X[t.reshape(X.shape[0],)==0, 0], X[t.reshape(X.shape[0],)==0, 1], "bs")
plt.plot(X[t.reshape(X.shape[0],)==1, 0], X[t.reshape(X.shape[0],)==1, 1], "g^")

plt.axis([-2, 2, -2, 2])
plt.show()

In [ ]:
# Entrenamos un autoencoder construido en keras
import keras

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X)

M=3
model = keras.models.Sequential([
    keras.layers.Dense(M, activation="relu", input_shape=X.shape[1:]),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, t, epochs=20000) # Calcula los pesos óptimos dado el training set X,t

In [ ]:
# Plot the decision boundary
x0, x1 = np.meshgrid(np.linspace(-2, 2, 100).reshape(-1, 1), np.linspace(-2, 2, 100).reshape(-1, 1),)
xfull = np.c_[x0.ravel(), x1.ravel()]
yfull = model.predict_proba(xfull)  # Precide la probabilidad de nuevos puntos

# Plot data set
zz = yfull.reshape(x0.shape)
plt.figure(figsize=(8, 6))

left_right = np.array([2.9, 7])

contour = plt.contour(x0, x1, zz, levels=[0.1, 0.5, 0.9], cmap=plt.cm.brg)
plt.clabel(contour, inline=1, fontsize=12)
plt.plot(X[t.reshape(X.shape[0],)==0, 0], X[t.reshape(X.shape[0],)==0, 1], "bs")
plt.plot(X[t.reshape(X.shape[0],)==1, 0], X[t.reshape(X.shape[0],)==1, 1], "g^")
plt.axis([-2, 2, -2, 2])
plt.show()

Al igual que en el ejemplo de regresión **el tiempo de entrenamiento se incrementa notablemente**. Como contrapartida **la búsqueda de características es automática**.

In [ ]:
# Guardar modelo
model_json = model.to_json()
with open("SvsT.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("SvsT.h5")
print("Saved model to disk")

In [ ]:
json_file = open('SvsT.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("SvsT.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_train, t, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

## Ejemplo completo: MNIST

Analizaremos ahora un ejemplo realista: la clasificación MNIST, al que le añadiremos la siguientes características:

- Early stopping y guardado y recuperación automáticos del modelo
- Evaluación final versus conjunto de test
- Clasificador multiclase basado en **softmax**
- Normalización de los datos

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
# print(mnist.DESCR) # Uncomment to show description
X, t = mnist["data"], mnist["target"] # N = 70000, D = 784 

from sklearn.utils import shuffle
X, t = shuffle(X, t, random_state=0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, t_train, t_test = X[:60000], X[60000:], t[:60000], t[60000:]
X_train, X_valid, t_train, t_valid = X_train[:50000], X_train[50000:], t_train[:50000], t_train[50000:] 

def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")
    
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.plasma, **options)
    plt.axis("off")

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=X.shape[1:]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.summary()

try: 
    model = keras.models.load_model("modelo.h5") # cargar modelo
except:
    pass

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("modelo.h5", save_best_only=True)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, t_train, epochs=50, batch_size=1000, validation_data=(X_valid, t_valid), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
print(history.params)

import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

#print(clf.best_params_)

print(f'The score (accurary) in the training set is {model.evaluate(X_train,t_train)[1]*100:.02f}%')
print(f'The score (accuracy) in the test set is {model.evaluate(X_test,t_test)[1]*100:.02f}%\n')

plot_digits(scaler.inverse_transform(X_test[:10]), images_per_row=10)
print('Labels predicted for the 10 first pictures in the test set')
print(model.predict_classes(X_test[:10]))
plt.show()